In [1]:
import sys,os,re,string
import csv
from itertools import combinations, chain, islice
from collections import Counter

import urllib
import numpy as np
import time
import pandas as pd

import json
import datetime
import pylab as pl

from pprint import pprint
from bs4 import BeautifulSoup as bs
import inspect



cmd_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile( inspect.currentframe() ))[0]) + 
                              "/word_cloud-master")
if cmd_folder not in sys.path:
     sys.path.insert(0, cmd_folder)


sys.path.append(os.getcwd() + "/word_cloud-master")
print (sys.path)
from wordcloud import make_wordcloud
from sklearn.feature_extraction.text import CountVectorizer
from config import tvsfile
from argparse import ArgumentParser


['/Users/fbianco/science/Dropbox/LSST_TVS/TVSaffiliations/word_cloud-master', '', '/usr/local/cuda/bin', '/usr/local/bin', '/usr/bin', '/bin', '/usr/sbin', '/sbin', '/opt/X11/bin', '/Library/TeX/texbin', '/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin', '/Applications/Xcode.app/Contents/Developer/usr/bin', '/usr/local/mysql/bin', '/Users/fbianco/science/Dropbox/randomutils', '/usr/texbin', '/Users/fbianco/science/Dropbox/LSST_TVS/TVSaffiliations', '//anaconda/lib/python27.zip', '//anaconda/lib/python2.7', '//anaconda/lib/python2.7/plat-darwin', '//anaconda/lib/python2.7/plat-mac', '//anaconda/lib/python2.7/plat-mac/lib-scriptpackages', '//anaconda/lib/python2.7/lib-tk', '//anaconda/lib/python2.7/lib-old', '//anaconda/lib/python2.7/lib-dynload', '//anaconda/lib/python2.7/site-packages', '//anaconda/lib/python2.7/site-packages/Sphinx-1.4.6-py2.7.egg', '//anaconda/lib/python2.7/site-packages/aeosa', '//anaconda/lib/python2.7/site-packages/matplotlib

In [2]:
def most_common(lst):
    return max(set(lst), key=lst.count)

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
class OrderedTuple(tuple):

    def __init__(self, init=()):
        if init:
            init = list(init)
            init.sort()
            init = tuple(init)
            self._oinit = init
            tuple.__init__(init)
        else:
            tuple.__init__(self)

    def __eq__(self, other):
        return self._oinit[0] == other._oinit[0] and self._oinit[1] == other._oinit[1]
        # return self._oinit == other._oinit

    def __hash__(self):
        return hash(self._oinit)

    def __repr__(self):
        return repr(self._oinit)

def mycounter(s):
    mydic = {}
    for w in s.split():
        #print w
#        print len(re.findall(w, s))
        if len(w)<3:
            continue
        if w in mydic.keys():
            mydic[w]=mydic[w]+1
        else:
            mydic[w]=1
#        try:
#            mydic[w] = len(re.findall(w,s))
#        except:
#            print  len(re.findall(w,s))
#            pass

    #print mydic
    return mydic

def parse_args(subglist):
    """ Use ArgParser to build up the arguments we will use in our script
    """
    stored_args = {}
    # get the script name without the extension & use it to build up
    # the json filename
    parser = ArgumentParser(description='Selecting members by subgroup')
    parser.add_argument('subgroup',
                        action='store',
                        default=None,
                        help='Choose the subgroup affiliation:' +
                        ' -- '.join([s for s in subglist]))

    args = parser.parse_args()

    return args


In [9]:
pl.ion()
print (tvsfile)
if tvsfile is None:
    print ("Required Argument: Google Doc file identifier (if you do not have it email federica!)")
    sys.exit()
    

TVSMembers = pd.read_csv('https://docs.google.com/spreadsheets/d/' +
                        tvsfile +
                        '/export?gid=0&format=csv',
                        index_col=0)
TVSMembers.head()
TVSMembers.tail()

1Q8YPGxh-Ebs02ykfGypj7oThQrgqKzrU6qyIuPT3bhY


,first name,affiliation,continent,email,primary,secondary,secondary.1,secondary.2,Unnamed: 9,Unnamed: 10,date added,added by
last name,,,,,,,,,,,,
Ciardi,David,Caltech,NaN,ciardi@ipac.caltech.edu,Transiting Planets,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jacklin,Savannah,Fisk University/ Vanderbilt University,NaN,savannah.jacklin@gmail.com,Transiting Planets,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Lund,Michael,Vanderbilt,NaN,mike.lund@gmail.com,Transiting Planets,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodriguez,Joseph,Vanderbilt,NaN,rodriguez.jr.joey@gmail.com,Transiting Planets,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tejos,Nicolas,Lick observatory,NaN,ntejos@ucolick.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
tvsgroups = TVSMembers.primary.unique()
subgroups = TVSMembers.primary
subgroups  = pd.concat([TVSMembers.primary, TVSMembers.secondary, TVSMembers['secondary.1'], TVSMembers['secondary.2']])

In [11]:
subgroups.size

848

In [12]:
#content = [[TVSMembers.primary.values[i]] for i in range(TVSMembers.size)]
len(TVSMembers.primary.values), len(TVSMembers.index)

(212, 212)

In [13]:
 
mywords=' '.join([sg.replace(' ', '').replace('/', '') for sg in subgroups if not isinstance(sg, float)])#[inner for outer in content for inner in outer ])
content = [[TVSMembers.primary.values[i], TVSMembers.secondary.values[i], 
            TVSMembers['secondary.1'].values[i], 
            TVSMembers['secondary.2'].values[i]] for i in range(len(TVSMembers.index))]
mycounts=mycounter(mywords)        
words=[]
counts=[]
for k in mycounts.iterkeys():
    words.append(k)
    counts.append(mycounts[k])
    
f = open('data/affiliationsNumbers.csv','w')
f.write("subg,tot\n")
for k in mycounts:
    f.write(k + ",%d\n"%mycounts[k])
f.write("totalaff" + ",%d\n"%np.array(mycounts.values()).sum())
f.write("totalmemb" + ",%d\n"%len(TVSMembers.index))
f.close()

In [14]:
words=np.array(words)
counts=np.array(counts)
# throw away some words, normalize
words = words[counts > 1]
counts = counts[counts > 1]
output_filename = "%s/cloud.png"%datetime.date.today()
os.system("mkdir %s"%datetime.date.today())
counts = make_wordcloud(words, counts, output_filename)

('DroidSansMono.ttf', 461)
('DroidSansMono.ttf', 460)
('DroidSansMono.ttf', 459)
('DroidSansMono.ttf', 458)
('DroidSansMono.ttf', 457)
('DroidSansMono.ttf', 456)
('DroidSansMono.ttf', 455)
('DroidSansMono.ttf', 454)
('DroidSansMono.ttf', 453)
('DroidSansMono.ttf', 452)
('DroidSansMono.ttf', 451)
('DroidSansMono.ttf', 450)
('DroidSansMono.ttf', 449)
('DroidSansMono.ttf', 448)
('DroidSansMono.ttf', 447)
('DroidSansMono.ttf', 446)
('DroidSansMono.ttf', 445)
('DroidSansMono.ttf', 444)
('DroidSansMono.ttf', 443)
('DroidSansMono.ttf', 442)
('DroidSansMono.ttf', 441)
('DroidSansMono.ttf', 440)
('DroidSansMono.ttf', 439)
('DroidSansMono.ttf', 438)
('DroidSansMono.ttf', 437)
('DroidSansMono.ttf', 436)
('DroidSansMono.ttf', 435)
('DroidSansMono.ttf', 434)
('DroidSansMono.ttf', 433)
('DroidSansMono.ttf', 432)
('DroidSansMono.ttf', 431)
('DroidSansMono.ttf', 430)
('DroidSansMono.ttf', 429)
('DroidSansMono.ttf', 428)
('DroidSansMono.ttf', 427)
('DroidSansMono.ttf', 426)
('DroidSansMono.ttf', 425)
(

In [15]:
_t = time.time()
wordlists={}
i=0
#print content
print (content)
for row in content:
    i+=1
    wordlists[i] = {}
    #print ([s for s in row if s != '' and not isinstance(s, float)])
    try:      
        thisset = set([s for s in row if s != '' and not isinstance(s, float)])
    except TypeError: 
        continue

    wordlists[i]['words'] = thisset
    _tuples = combinations(wordlists[i]['words'], 2)
    tuples = set()
    for t in _tuples:
        tuples.add(OrderedTuple(t))
    wordlists[i]['tuples'] = tuples # unique ordered tuples
    wordlists[i]['hashes'] = set([ hash(t) for t in tuples ])
    assert len(tuples) == len(wordlists[i]['hashes'])
    # print('%s, %s' % (user, len(tuples)))
delta = time.time()-_t
print('\n%d data collected in %dsec.' % (i, delta))

[['Microlensing', 'Fast Transients', 'Galactic', 'Transiting Planets'], ['Multiwavelength Characterization/Counterparts', 'Fast Transients', 'Cosmological', 'Tidal Disruption Events'], ['Multiwavelength Characterization/Counterparts', 'Galactic', 'Transiting Planets', 'Interacting Binaries'], ['Tidal Disruption Events', 'Fast Transients', 'Multiwavelength Characterization/Counterparts', 'Tidal Disruption Events'], ['Classification/Characterization', 'Fast Transients', 'Multiwavelength Characterization/Counterparts', 'Non-degenerate Eruptive Variables'], ['Microlensing', 'Fast Transients', 'Distance Scales', 'Cosmological'], ['Classification/Characterization', 'Fast Transients', 'Multiwavelength Characterization/Counterparts', 'Distance Scales'], ['Supernovae', 'Fast Transients', 'Tidal Disruption Events', 'Classification/Characterization'], ['Classification/Characterization', 'Fast Transients', 'Galactic', 'Pulsating Variables'], ['Classification/Characterization', 'Distance Scales', '

In [16]:
wordlists

{1: {'hashes': {-1210201123893767357,
   -10194061194384049,
   2611239757624509459,
   2749125329368756551,
   3889909371238269627,
   4434328516039651519},
  'tuples': {('Fast Transients', 'Galactic'),
   ('Fast Transients', 'Microlensing'),
   ('Fast Transients', 'Transiting Planets'),
   ('Galactic', 'Microlensing'),
   ('Galactic', 'Transiting Planets'),
   ('Microlensing', 'Transiting Planets')},
  'words': {'Fast Transients',
   'Galactic',
   'Microlensing',
   'Transiting Planets'}},
 2: {'hashes': {-7460472462923700992,
   -2901701964653590550,
   -62737503984141675,
   476394684546984696,
   1073403636413331490,
   8809723922306783285},
  'tuples': {('Cosmological', 'Tidal Disruption Events'),
   ('Cosmological', 'Fast Transients'),
   ('Fast Transients', 'Multiwavelength Characterization/Counterparts'),
   ('Fast Transients', 'Tidal Disruption Events'),
   ('Cosmological', 'Multiwavelength Characterization/Counterparts'),
   ('Multiwavelength Characterization/Counterparts',

In [17]:
import itertools
tmp = list(itertools.combinations(content[0], 2))
for row in content[1:]:
    tmp = tmp + list(itertools.combinations(row, 2))
print tmp

[('Microlensing', 'Fast Transients'), ('Microlensing', 'Galactic'), ('Microlensing', 'Transiting Planets'), ('Fast Transients', 'Galactic'), ('Fast Transients', 'Transiting Planets'), ('Galactic', 'Transiting Planets'), ('Multiwavelength Characterization/Counterparts', 'Fast Transients'), ('Multiwavelength Characterization/Counterparts', 'Cosmological'), ('Multiwavelength Characterization/Counterparts', 'Tidal Disruption Events'), ('Fast Transients', 'Cosmological'), ('Fast Transients', 'Tidal Disruption Events'), ('Cosmological', 'Tidal Disruption Events'), ('Multiwavelength Characterization/Counterparts', 'Galactic'), ('Multiwavelength Characterization/Counterparts', 'Transiting Planets'), ('Multiwavelength Characterization/Counterparts', 'Interacting Binaries'), ('Galactic', 'Transiting Planets'), ('Galactic', 'Interacting Binaries'), ('Transiting Planets', 'Interacting Binaries'), ('Tidal Disruption Events', 'Fast Transients'), ('Tidal Disruption Events', 'Multiwavelength Character

In [20]:
iterators = []
for user in wordlists.iterkeys():
    iterators.append(iter(wordlists[user]['tuples']))
#print (iterators)
hash_dict = {}
# { hash_value: OrderTuple, ... }
print('Making hash_dict ...')
_t=time.time()
i=0
for _tuple in chain(*iterators):
    i+=1
    hash_dict[hash(_tuple)] = _tuple
    if (i%100000 == 0):
        sys.stdout.write('\r%d' % i)
        sys.stdout.flush()
print('\n%d hash_dict done in %dsec.' % (i, delta))
    

iterators = []
for user in wordlists.iterkeys():
    iterators.append(iter(wordlists[user]['hashes']))
_stats = {}
done = []
i=0

print('Counting ...')
_t = time.time()
for t in chain(*iterators):
    if t in done:
        continue
    i+=1
    if (i%1000 == 0):
        sys.stdout.write("\r%s" % i)
        sys.stdout.flush()
    done.append(t)
    _count = 0
    for user in wordlists.iterkeys():
        if t in wordlists[user]['hashes']:
            _count += 1
    _stats[hash(t)]=_count

delta = time.time() - _t
print('\nDone writing %d in %dsec.' % (i, delta))

print('Preparing data to write ... ')
stats = []
error_hashes = []
for _hash, _tuple in hash_dict.iteritems():
    try:
        _count = _stats[_hash]
    except KeyError as e:
        error_hashes.append(e)
    else:
        stats.append((_tuple, _count))
print('error_hashes=%r' % len(error_hashes))

Making hash_dict ...

639 hash_dict done in 0sec.
Counting ...

Done writing 89 in 0sec.
Preparing data to write ... 
error_hashes=0


In [21]:
alllist = set(sum([[stat[0][0],stat[0][1]] for stat in stats if  stat[1]>=2],[]))
allpairs = list(itertools.combinations(list(alllist), 2))

In [22]:
allstats = {}

for stat in stats:
    print (stat, allstats.keys())
    if 1: #not stat[0][0] in allstats.keys():
        allstats[stat[0][0]] = {}
        allstats[stat[0][0]][stat[0][1]] = stat[1]
        #if 'star' in stat[0][0]:
        #    print allstats[stat[0][0]]
#       for i in range(stat[1]):

#    raw_input()
print (allstats)

((('Fast Transients ', 'Multiwavelength Characterization/Counterparts'), 1), [])
((('Cosmological', 'Microlensing'), 3), ['Multiwavelength Characterization/Counterparts'])
((('Microlensing', 'Transiting Planets'), 6), ['Multiwavelength Characterization/Counterparts', 'Microlensing'])
((('Blazars', 'Fast Transients'), 1), ['Multiwavelength Characterization/Counterparts', 'Microlensing'])
((('Cosmological', 'Tidal Disruption Events'), 2), ['Fast Transients', 'Multiwavelength Characterization/Counterparts', 'Microlensing'])
((('Magnetically Active Stars', 'Transiting Planets'), 2), ['Fast Transients', 'Multiwavelength Characterization/Counterparts', 'Microlensing', 'Tidal Disruption Events'])
((('Fast Transients', 'Supernovae'), 31), ['Fast Transients', 'Multiwavelength Characterization/Counterparts', 'Magnetically Active Stars', 'Microlensing', 'Tidal Disruption Events'])
((('Interacting Binaries', 'Pulsating Variables'), 10), ['Fast Transients', 'Multiwavelength Characterization/Counter

In [23]:
alllist = set(sum([[stat[0][0],stat[0][1]] for stat in stats if  stat[1]>=2],[]))
print alllist,len(alllist)
mymatrix=np.zeros((len(alllist),len(alllist)),int)
print mymatrix.shape
import csv


set(['Interacting Binaries', 'Galactic', 'Supernovae', 'Cosmological', 'Multiwavelength Characterization/Counterparts', 'Tidal Disruption Events', 'Non-degenerate Eruptive Variables', 'Fast Transients', 'Microlensing', 'Classification/Characterization', 'Magnetically Active Stars', 'Distance Scales', 'Pulsating Variables', 'Transiting Planets', 'Blazars']) 15
(15, 15)


In [24]:
for f in allpairs: 
    print tmp.count(f), tmp.count(f[::-1])

5 3
3 4
0 0
9 4
1 1
3 2
5 8
0 0
7 2
1 1
1 0
8 2
3 3
2 1
4 0
0 4
6 6
0 2
1 4
1 11
2 4
5 10
1 2
2 5
5 4
7 1
0 0
7 7
11 28
6 8
5 3
14 18
2 2
11 13
0 0
1 5
3 3
0 2
0 0
8 8
1 1
0 0
5 3
2 1
6 3
0 0
6 2
0 3
0 0
0 0
13 9
3 4
17 31
3 2
12 19
0 2
2 2
2 1
6 1
4 2
1 0
1 11
0 0
4 3
0 0
0 0
0 2
0 0
1 1
2 7
1 1
5 2
0 0
0 2
4 1
1 0
0 0
2 4
7 17
0 0
2 1
4 1
3 1
1 0
4 1
0 0
1 1
1 0
5 2
0 0
1 1
4 5
7 6
7 5
1 0
0 0
1 1
1 1
0 0
6 8
1 1
0 0
2 4
0 0
1 0


In [27]:
with open('data/affiliations.csv', 'wb') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=' ', quotechar=' ',quoting=csv.QUOTE_NONE,escapechar='\\')
        spamwriter.writerow(['word1,word2,count'])
        for i,word1 in enumerate(alllist):
            for j,word2 in  enumerate(alllist):
                if word1==word2:
                    mymatrix[i][j]=0
                    spamwriter.writerow( ['"%s","%s",%d'%(word1.replace(" ",""),word2.replace(" ",""),0)])
#                    spamwriter.writerow( ['"%s","%s",%d,%d'%(word1,word2,0,subjects[word1])])
                else:
                    
                    cnt = tmp.count((word1,word2)) +  tmp.count((word2,word1))
                    #print ('%s,%s,%s' %(word1, word2, cnt))
                    try: 
                        print (cnt)
                    except: 
                        print ("not there")
                        pass
                    try:  
                        mymatrix[i][j] = cnt
                        spamwriter.writerow( ['"%s","%s",%d'%(word1.replace(" ",""), word2.replace(" ",""), cnt)])
                    except: 
                        mymatrix[i][j]=0
                        spamwriter.writerow( ['"%s","%s",%d,'%(word1.replace(" ",""), word2.replace(" ",""), 0)])


#        for stat in stats:
#            if stat[0][0] not in  commonastrowords and not is_number(stat[0][0]) and stat[0][1] not in   
#commonastrowords and not is_number(stat[0][1]) and len(stat[0][0])>=3 and len(stat[0][1])>=3  and stat[1]>1:


#    for word1 in alllist:
#        print word1,
#    for  i,word1 in enumerate(alllist):
#        print word1,"\t\t[" ,
#        for j,word2 in enumerate(alllist):
#            print mymatrix[i][j],",",
#        print "],"

#    print('Writing data')
#    with open('stats.json', 'w') as fo:
#        fo.write(json.dumps(stats))
print('DONE.')

8
7
0
13
2
5
13
0
9
2
1
10
6
3
8
4
4
12
2
5
12
6
15
3
7
9
8
0
7
4
14
39
14
8
32
4
24
0
6
6
2
0
0
4
14
16
2
0
8
3
9
0
8
3
0
0
13
12
39
16
22
7
48
5
31
2
4
3
7
6
2
2
14
2
22
1
12
0
7
0
0
2
0
2
5
5
8
0
7
1
9
2
7
0
2
5
1
0
13
12
32
8
48
12
9
6
24
0
3
5
4
1
0
6
4
3
5
0
2
6
5
0
2
1
7
0
9
15
24
9
31
7
7
24
5
2
9
13
12
1
2
3
0
0
2
0
0
0
0
2
0
2
2
0
1
7
6
8
4
0
2
3
2
9
0
14
2
0
10
9
6
3
3
2
5
5
1
13
2
14
6
0
6
8
2
0
7
0
1
4
7
12
2
2
6
1
3
0
0
0
6
2
0
1
0
1
0
0
0
1
DONE.
